# Dependencies loading

In [1]:
#libraries
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning


np.random.seed(2024)
pd.set_option("display.max_columns", 500)

## Data loading

In [9]:
FE_data_train = pd.read_csv('../input_data/fe_X_train.csv',index_col=0)
FE_y_train = pd.read_csv('../input_data/fe_y_train.csv',index_col=0)
RE_data_train = pd.read_csv('../input_data/re_X_train.csv',index_col=0)
RE_y_train = pd.read_csv('../input_data/re_y_train.csv',index_col=0)
fe_results = pd.read_csv('../input_data/fe_results.csv',index_col=0)

In [10]:
FE_data_train

,Customer_Age,Dependent_count,Education_Level,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Education_Level_NA,Income_Category_NA,Gender_M,MS_Married,MS_Single,ln_Total_Trans_Amt,ln_Credit_Limit,ln_Avg_Transaction,Credit_utilization
0,27.0,0.0,3.0,2.0,0.0,36.0,5.0,1.0,0.0,13439.0,2003.0,11436.0,0.936,4715.0,86.0,0.720,0.149,0.0,1.0,True,True,False,8.458504,9.505916,4.004157,0.149044
1,46.0,0.0,0.0,2.0,1.0,36.0,6.0,3.0,2.0,30501.0,2030.0,28471.0,0.560,2525.0,45.0,0.500,0.067,0.0,0.0,False,False,True,7.833996,10.325515,4.027334,0.066555
2,46.0,2.0,1.0,0.0,0.0,37.0,3.0,1.0,1.0,6118.0,1092.0,5026.0,0.757,4434.0,81.0,0.620,0.178,0.0,0.0,True,True,False,8.397057,8.718991,4.002608,0.178490
3,57.0,2.0,3.0,0.0,0.0,45.0,6.0,2.0,2.0,1438.3,740.0,698.3,0.698,4970.0,87.0,0.933,0.514,0.0,0.0,False,True,False,8.511175,7.271217,4.045267,0.514496
4,56.0,1.0,2.0,3.0,0.0,36.0,3.0,3.0,3.0,34516.0,1477.0,33039.0,1.234,2395.0,50.0,0.724,0.043,0.0,0.0,True,True,False,7.781139,10.449178,3.869116,0.042792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8096,65.0,1.0,0.0,0.0,0.0,55.0,3.0,2.0,4.0,8963.0,0.0,8963.0,0.744,4511.0,76.0,0.689,0.000,0.0,0.0,False,False,False,8.414274,9.100860,4.083541,0.000000
8097,40.0,1.0,1.0,0.0,0.0,30.0,6.0,2.0,1.0,2579.0,1959.0,620.0,0.618,4300.0,69.0,0.683,0.760,0.0,0.0,False,True,False,8.366370,7.855157,4.132264,0.759597
8098,52.0,2.0,4.0,3.0,1.0,48.0,6.0,6.0,1.0,34516.0,1473.0,33043.0,0.642,4254.0,73.0,0.587,0.043,1.0,0.0,False,False,False,8.355615,10.449178,4.065156,0.042676
8099,44.0,4.0,3.0,1.0,0.0,40.0,6.0,5.0,3.0,1572.0,874.0,698.0,0.947,4996.0,80.0,0.739,0.556,0.0,0.0,False,True,False,8.516393,7.360104,4.134366,0.555980


In [11]:
FE_y_train

,Attrition_Flag
6150,0
4368,0
4175,0
7352,0
1285,0
...,...
4753,0
5370,0
4621,0
6774,0


In [12]:
RE_data_train

,Customer_Age,Dependent_count,Education_Level,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Education_Level_NA,Income_Category_NA,Gender_M,MS_Married,MS_Single,ln_Total_Trans_Amt,ln_Credit_Limit,ln_Avg_Transaction,Credit_utilization
0,27.000000,0.000000,3.000000,2.000000,0.00000,36.000000,5.000000,1.000000,0.000000,13439.000000,2003.000000,11436.000000,0.936000,4715.000000,86.000000,0.720000,0.149000,0.0,1.000000,True,True,False,8.458504,9.505916,4.004157,0.149044
1,46.000000,0.000000,0.000000,2.000000,1.00000,36.000000,6.000000,3.000000,2.000000,30501.000000,2030.000000,28471.000000,0.560000,2525.000000,45.000000,0.500000,0.067000,0.0,0.000000,False,False,True,7.833996,10.325515,4.027334,0.066555
2,46.000000,2.000000,1.000000,0.000000,0.00000,37.000000,3.000000,1.000000,1.000000,6118.000000,1092.000000,5026.000000,0.757000,4434.000000,81.000000,0.620000,0.178000,0.0,0.000000,True,True,False,8.397057,8.718991,4.002608,0.178490
3,57.000000,2.000000,3.000000,0.000000,0.00000,45.000000,6.000000,2.000000,2.000000,1438.300000,740.000000,698.300000,0.698000,4970.000000,87.000000,0.933000,0.514000,0.0,0.000000,False,True,False,8.511175,7.271217,4.045267,0.514496
4,56.000000,1.000000,2.000000,3.000000,0.00000,36.000000,3.000000,3.000000,3.000000,34516.000000,1477.000000,33039.000000,1.234000,2395.000000,50.000000,0.724000,0.043000,0.0,0.000000,True,True,False,7.781139,10.449178,3.869116,0.042792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13593,43.412261,2.396935,1.603065,3.301533,0.00000,33.206130,2.507663,2.000000,4.000000,26377.957089,0.000000,26377.957089,0.567925,2095.393487,45.095402,0.627923,0.000000,0.0,0.698467,False,True,False,7.647408,10.180247,3.839102,0.000000
13594,43.051042,4.978125,0.000000,0.000000,0.00000,36.036459,4.978125,3.000000,3.985416,2689.452089,2383.044779,306.407310,0.365963,1942.217724,52.905207,0.654724,0.886504,0.0,0.007292,False,True,True,7.571560,7.897091,3.603320,0.886082
13595,46.616168,2.097795,0.601470,1.000000,0.00000,42.014698,2.699265,3.202940,2.000000,3631.537590,334.248724,3297.288866,0.558371,2069.645563,33.706614,0.664179,0.090205,0.0,0.699265,True,False,True,7.635039,8.197015,4.124397,0.090386
13596,39.154870,2.309739,0.000000,3.845130,0.84513,21.632782,2.619478,1.309739,2.000000,34516.000000,761.309228,33754.690772,0.873368,7866.267608,75.929217,0.730497,0.022071,0.0,0.000000,False,False,False,8.969456,10.449178,4.640049,0.022057


In [13]:
RE_y_train

,Attrition_Flag
0,0
1,0
2,0
3,0
4,0
...,...
13593,1
13594,1
13595,1
13596,1


In [14]:
fe_results

,Variable,Variance,Chi2,p_Chi2,F_S,M_I,EN_coef
0,Customer_Age,64.11,2.705,0.100,1.958,0.004,-2.387532e-04
1,Dependent_count,1.69,2.431,0.119,3.373,0.000,1.009164e-02
2,Education_Level,1.83,5.758,0.016,6.524,0.000,3.303844e-03
3,Income_Category,1.78,0.985,0.321,0.757,0.000,-5.787566e-03
4,Card_Category,0.11,0.004,0.949,0.003,0.000,1.808591e-02
5,Months_on_book,63.90,1.552,0.213,0.874,0.006,-4.020327e-04
6,Total_Relationship_Count,2.43,107.034,0.000,171.491,0.007,-4.003024e-02
7,Months_Inactive_12_mon,1.02,83.554,0.000,195.501,0.022,3.944177e-02
8,Contacts_Count_12_mon,1.23,180.411,0.000,377.836,0.028,4.368884e-02
9,Credit_Limit,83066683.46,23895.053,0.000,2.487,0.002,2.255557e-06
